## Подключение к mssql server

In [1]:
import pyodbc

In [2]:
import pandas as pn

In [3]:
#функция проверки, является ли строка числом
def is_digit(string):
    if string.isdigit():
        return True
    else:
        try:
            float(string)
            return True
        except ValueError:
            return False
        
def make_int(string):
    if string.isdigit:
        return string
    else:
        return '0'

def make_float(string):
    

In [3]:
driver = 'DRIVER={SQL Server}'
server = 'SERVER=localhost'
port = 'PORT=1433'
db = 'DATABASE=trash'
user = 'UID=trash'
pw = 'PWD=password'
conn_str = ';'.join([driver, server, port, db, user, pw])

In [4]:
conn = pyodbc.connect(conn_str)
cursor = conn.cursor() 

In [5]:

cursor.execute('select * from trash.sources_catalog')
row = cursor.fetchone()
rest_of_rows = cursor.fetchall()

In [14]:
# получить список дат с 92 по 2018 год
query = """select frm104, frm112
    from sources.t_days
    where dat between '19920101' and '19980101'
    order by dat desc"""


In [15]:
df_date = pn.read_sql(query,conn)

In [9]:
from lxml import html

In [16]:
for index, row in df_date.iterrows():
    url = 'http://cbr.ru/currency_base/daily/?date_req={}'.format(row["frm104"])
    page = html.parse(url)
    e = page.getroot().find_class('data').pop()
    t = e.getchildren().pop()
    
    query = """DELETE FROM trash.sources_cbr_currency_daily_stg WHERE DAT = '{}';""".format(row["frm112"])
    cursor.execute(query)

    for element in t[1:]:
        dat = row["frm104"]
        digit_code = int(element.getchildren()[0].text_content())
        charcode = element.getchildren()[1].text_content()
        units_amount = int(element.getchildren()[2].text_content())
        currency_name = element.getchildren()[3].text_content()
        exchange_rate = element.getchildren()[4].text_content().replace(' ', '').replace(',', '.')
        
        query = """
            INSERT INTO trash.sources_cbr_currency_daily_stg(dat, digit_code, charcode, units_amount, currency_name, exchange_rate) 
            VALUES('{}', {}, '{}', {}, '{}', {});
            """.format(dat, \
               , digit_code \
               , charcode \
               , units_amount \
               , currency_name \
               , exchange_rate)
        cursor.execute(query)
    conn.commit()
    

ValueError: invalid literal for int() with base 10: ' — '

In [17]:
url

'http://cbr.ru/currency_base/daily/?date_req=31.08.1996'

In [19]:
element.getchildren()[1].text_content()

'\r\n      '

In [10]:
rest_of_rows

[]

In [21]:
cursor.close()
conn.close()

ProgrammingError: The cursor's connection has been closed.